In [19]:
%matplotlib inline
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask
import datetime as dt
import multiprocessing
import folium
import branca
import branca.colormap as cm
import pylab
import random as rd
import geopandas
import seaborn
import time
import matplotlib.colors as mcolors

from datetime import datetime, date
from shapely.geometry import Point, Polygon, MultiPolygon
from tqdm.notebook import tqdm
from collections import Counter
from folium.plugins import HeatMap
from folium.features import DivIcon
from folium.plugins import HeatMapWithTime

# Single machine progress bar
from dask.diagnostics import ProgressBar

pd.options.display.precision = 2
#pd.set_option('display.max_columns', None)
nbRowsToDisplay = 100
pd.options.display.min_rows = nbRowsToDisplay
pd.options.display.max_rows = nbRowsToDisplay

plt.rcParams["figure.figsize"] = (20,10)

In [20]:
def plotHistogram(counts,labels,tit,xlab,ylab,width=0.8):
    fig = pylab.figure(figsize=(20,10))    
    ax = fig.add_subplot(1,1,1)
    ax.bar(labels, counts, width,align='center')
    ax.set_title(tit, fontsize=18)
    ax.set_xlabel(xlab, fontsize=18)
    ax.set_ylabel(ylab, fontsize=18)
    ax.grid(True)
    ax.set_axisbelow(True)

In [21]:
colorList=list(mcolors.CSS4_COLORS.keys())

def addPointsToMap(my_map,latitudes,longitudes,colors,opacity=0.3):
    
    #Add point
    for lat,lon,col in zip(latitudes,longitudes,colors):
        folium.CircleMarker([lat, lon],
                            radius=1,fill=True,color=col,opacity=opacity
                           ).add_to(my_map)
    
    return my_map

In [22]:
#Define a function to display an heatmap
Cph_Lat = 12.5683
Cph_Long = 55.6761

#Create a gradient map
steps = 20
#color_map=cm.linear.Accent_07.scale(0,1).to_step(steps)
color_map = cm.LinearColormap(['blue', 'yellow', 'red'],vmin=0, vmax=1)
#color_map1 = cm.LinearColormap(['blue', 'yellow', 'red'],vmin=0, vmax=120)

gradient_map={}
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

def generateBaseMap(default_location=[Cph_Long, Cph_Lat], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=12,tiles=None)
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(base_map)
    return base_map

def plotHeatmap(latitude,longitude,avgDemand):
    
    heatmap = generateBaseMap() #Base map
    #Heat map
    heatmap.add_child(HeatMap(data=np.vstack((latitude, longitude,avgDemand)).T.tolist(), 
                              radius=7, max_zoom=5,  gradient=gradient_map))
    
    color_map.add_to(heatmap) #Color bar
    
    return heatmap

def plotHeatmapWithTime(data,index):
    
    heatmap = generateBaseMap() #Base map
    
    #Heatmap
    heatmap.add_child(HeatMapWithTime(data, radius=5,index=index,  gradient=gradient_map))
    
    color_map.add_to(heatmap) #Color bar
    
    return heatmap

In [23]:
fileDirectory = '/mnt/sdb1/gauthier/jupyter/'

# Assess the demand by hour

In [15]:
df=pd.read_pickle(fileDirectory+'data/Simulation/DistanceFromCSAndDemandByHour_150000iterations_Sept_penetrationRate=10%')
maxDemandByHour=50 #Maximum 50kWh can be asked by hour as it is the maximum power
hours=list(range(24))
for h in hours: 
    df['scaledDemandAt'+str(h)]=df['demandAt'+str(h)]/maxDemandByHour

In [16]:
maxDemandByHour=50 #Maximum 50kWh can be asked by hour as it is the maximum power
hours=list(range(24))
for h in hours: 
    df['scaledDemandAt'+str(h)]=df['demandAt'+str(h)]/maxDemandByHour

In [17]:
#Check how much of the demand is kept by filtering out non-significant demand
for h in hours:
    print('% scaled demand higher than 0.01:',df[df['scaledDemandAt'+str(h)]>0.01]['demandAt'+str(h)].sum()/\
          df['demandAt'+str(h)].sum())
    #print("% data kept:",len(df[df['scaledDemandAt'+str(h)]>0.01])/len(df))

% scaled demand higher than 0.01: 0.9928418383828579
% scaled demand higher than 0.01: 0.9934947225856482
% scaled demand higher than 0.01: 0.993134163103661
% scaled demand higher than 0.01: 0.9901513574055151
% scaled demand higher than 0.01: 0.9879674460221937
% scaled demand higher than 0.01: 0.977702844799438
% scaled demand higher than 0.01: 0.98074668386271
% scaled demand higher than 0.01: 0.979350201251315
% scaled demand higher than 0.01: 0.9759505520376278
% scaled demand higher than 0.01: 0.9778929540726866
% scaled demand higher than 0.01: 0.9744868688894629
% scaled demand higher than 0.01: 0.9692216464206296
% scaled demand higher than 0.01: 0.9659444316381943
% scaled demand higher than 0.01: 0.9672714902989908
% scaled demand higher than 0.01: 0.9691787926670268
% scaled demand higher than 0.01: 0.9686710613366099
% scaled demand higher than 0.01: 0.9698280898690493
% scaled demand higher than 0.01: 0.9754461480144591
% scaled demand higher than 0.01: 0.979497312211243

In [29]:
#Visualize the demand by hour
data=[]

#hours=list(range(24))
hours=[8,17]
for h in hours: 
    
    #Only select the points with high enough unserved demand as the other points don't represent much unserved demand
    df1=df[df['scaledDemandAt'+str(h)]>0.01]
    
    data.append(np.vstack((df1.endLatitude, df1.endLongitude,df1['scaledDemandAt'+str(h)])).T.tolist())
    
heatmapWithTime = plotHeatmapWithTime(data,hours)

display(heatmapWithTime)